# `Catalog-To-Xpublish` Demo

**Author:** [Xavier R Nogueira](https://github.com/xaviernogueira)

**Overview:** This notebook demonstrates how to use the `Catalog-To-Xpublish` package to convert a STAC catalog of Zarr datasets into a [`xpublish`](https://xpublish.readthedocs.io/en/latest/) powered FastAPI server application. 

**Contents:**
1. [Build application](#Build-application)
2. [Test default endpoints](#Test-default-endpoints)
3. [Use OPeNDAP endpoints](#Use-OPeNDAP-endpoints)
4. [Test OPeNDAP clients](#Test-OPeNDAP-clients)


In [1]:
# do imports
import sys
import time
import json
from IPython.display import HTML
import hvplot.xarray
import subprocess
import requests
import xarray as xr
import netCDF4 as nc
from pathlib import Path

## Build application

In [2]:
# import inputs from run_server.py and spin up server
sys.path.append(str(Path.cwd().parent))
from examples.run_server import (
    CATALOG_TYPE,
    CATALOG_PATH,
    APP_NAME,
    XPUBLISH_PLUGINS,
    CONFIG_LOGGING_DICT,
    LOCAL_HOST,
    LOCAL_PORT,
)

[2023-05-30T15:45:02] INFO - Spinning up server from stac catalog at c:\Users\xrnogueira\Documents\Catalog-To-Xpublish\test_catalogs\test_stac_zarr_catalog.json.
[2023-05-30T15:45:03] INFO - Mounting a Xpublish server @ /conus404-daily to the main application.
[2023-05-30T15:45:03] INFO - Returning successfully created server application!


Code being executed in this section is shown below:

```python
catalog_to_xpublish.create_app(
    catalog_path=CATALOG_PATH,
    catalog_type=CATALOG_TYPE,
    app_name=APP_NAME,
    xpublish_plugins=XPUBLISH_PLUGINS,
    config_logging_dict=CONFIG_LOGGING_DICT,
)
```

Where the inputs are as follows:

In [3]:
# display inputs
INPUTS_DICT = {
    'catalog_type': CATALOG_TYPE,
    'catalog_path': CATALOG_PATH,
    'app_name': APP_NAME,
    'xpublish_plugins': XPUBLISH_PLUGINS,
    'config_logging_dict': CONFIG_LOGGING_DICT,
    'local_host': LOCAL_HOST,
    'local_port': LOCAL_PORT,
}
display(INPUTS_DICT)

{'catalog_type': 'stac',
 'catalog_path': WindowsPath('c:/Users/xrnogueira/Documents/Catalog-To-Xpublish/test_catalogs/test_stac_zarr_catalog.json'),
 'app_name': 'Xpublish Server',
 'xpublish_plugins': [xpublish_opendap.plugin.OpenDapPlugin],
 'config_logging_dict': {'level': 'INFO', 'log_file': 'xpublish_server.log'},
 'local_host': '127.0.0.1',
 'local_port': 8000}

In [4]:
# run the server (wait 15 seconds for it to start)
server_process = subprocess.Popen(['python', 'run_server.py'])
time.sleep(15)
server_process

<Popen: returncode: None args: ['python', 'run_server.py']>

## Test default endpoints

In [5]:
# set up base url
base_url = f'http://{LOCAL_HOST}:{LOCAL_PORT}'

# test that server is running
try:
    assert str(requests.get(base_url).json()) == "{'detail': 'Not Found'}"
    print('Server is running!')
except AssertionError:
    print('Server is not running!')

Server is running!


### Root catalog level (no datasets)

One can use `/json` at any level of the catalog to see a JSON representation of the catalog (in this case a STAC).

In [6]:
json.loads(requests.get(base_url + '/json').json())

{'type': 'Catalog',
 'id': 'nhgf-stac-catalog',
 'stac_version': '1.0.0',
 'description': 'Sample NHGF STAC catalog.',
 'links': [{'rel': 'self',
   'href': 'c:\\Users\\xrnogueira\\Documents\\Catalog-To-Xpublish\\test_catalogs\\test_stac_zarr_catalog.json',
   'type': 'application/json'},
  {'rel': 'root',
   'href': 'https://code.usgs.gov/wma/nhgf/stac/raw/main/catalog/catalog.json',
   'type': 'application/json'},
  {'rel': 'child',
   'href': 'https://code.usgs.gov/wma/nhgf/stac/raw/main/catalog/conus404-daily/collection.json',
   'type': 'application/json'}]}

To see if there are any "sub-catalogs" nested within a certain level of the catalog, one can use the `/catalogs` endpoint.

In [7]:
requests.get(base_url + '/catalogs').json()

['conus404-daily']

At any point one can see what the parent catalog is using `/parent_catalog` (in this case we are at the root).

In [8]:
requests.get(base_url + '/parent_catalog').json()

'This is the root catalog'

### Sub-catalog level (w/ datasets)

Next we navigate to the 'conus404-daily' sub-catalog and explore the endpoints there. To do so, one can simply append `/{SUB_CATALOG_NAME}` to the root URL.

If a sub-catalog has datasets, one can see what those datasets are using the `/datasets` endpoint.

In [9]:
requests.get(base_url + '/conus404-daily' + '/datasets').json()

['zarr-s3']

Same as the previous examples at the root level, we can query the sub-catalog level using `/catalogs`, `/parent_catalog`, and `/json`.

In [10]:
requests.get(base_url + '/conus404-daily' + '/catalogs').json()

[]

In [11]:
requests.get(base_url + '/conus404-daily' + '/parent_catalog').json()

'/'

Here we can see that the "zarr-s3" dataset has metadata included in the STAC catalog.

In [12]:
json.loads(requests.get(base_url + '/conus404-daily' + '/json').json())

{'type': 'Collection',
 'id': 'conus404-daily',
 'stac_version': '1.0.0',
 'description': 'CONUS404 40 years of daily values for subset of model output variables derived from hourly values on cloud storage',
 'links': [{'rel': 'root',
   'href': '../catalog.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': 'https://code.usgs.gov/wma/nhgf/stac/raw/main/catalog/conus404-daily/collection.json',
   'type': 'application/json'},
  {'rel': 'parent',
   'href': 'c:\\Users\\xrnogueira\\Documents\\Catalog-To-Xpublish\\test_catalogs\\test_stac_zarr_catalog.json',
   'type': 'application/json'}],
 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json',
  'https://stac-extensions.github.io/xarray-assets/v1.0.0/schema.json',
  'https://stac-extensions.github.io/datacube/v2.2.0/schema.json'],
 'cube:dimensions': {'time': {'type': 'temporal',
   'description': None,
   'extent': ['1979-10-01T00:00:00Z', '2021-09-25T00:00:00Z'],
   'step': 'P1DT0H0M0S'},
  'x': 

### `Xpublish` endpoints

`Xpublish` includes some default endpoints that are available when datasets are present.
* `datasets/{dataset_id}` - returns the HTML representation of the dataset.
* `datasets/{dataset_id}/keys` - returns variable names of the dataset.
* `datasets/{dataset_id}/info` - returns dataset info.
* `datasets/{dataset_id}/dict` - returns the dataset as a dictionary (won't be tested, very slow).

In [13]:
dataset_url = base_url + '/conus404-daily' + '/datasets' + '/zarr-s3'
display(HTML(requests.get(dataset_url).text))

In [14]:
display(requests.get(dataset_url + '/keys').json())

['ACDEWC',
 'ACDRIPR',
 'ACDRIPS',
 'ACECAN',
 'ACEDIR',
 'ACETLSM',
 'ACETRAN',
 'ACEVAC',
 'ACEVB',
 'ACEVC',
 'ACEVG',
 'ACFROC',
 'ACFRZC',
 'ACGHB',
 'ACGHFLSM',
 'ACGHV',
 'ACINTR',
 'ACINTS',
 'ACLHFLSM',
 'ACLWDNB',
 'ACLWUPB',
 'ACMELTC',
 'ACPONDING',
 'ACQLAT',
 'ACQRF',
 'ACRAINLSM',
 'ACRAINSNOW',
 'ACRUNSB',
 'ACRUNSF',
 'ACSHFLSM',
 'ACSNBOT',
 'ACSNFRO',
 'ACSNOM',
 'ACSNOWLSM',
 'ACSNSUB',
 'ACSUBC',
 'ACSWDNB',
 'ACSWDNLSM',
 'ACSWDNT',
 'ACSWUPB',
 'ACSWUPLSM',
 'ACTHROR',
 'ACTHROS',
 'ACTR',
 'ALBEDO',
 'BF',
 'BH',
 'C1F',
 'C1H',
 'C2F',
 'C2H',
 'C3F',
 'C3H',
 'C4F',
 'C4H',
 'CANWAT',
 'CF1',
 'CF2',
 'CF3',
 'CFN',
 'CFN1',
 'CLAT',
 'COSALPHA',
 'DN',
 'DNW',
 'DZS',
 'E',
 'E2',
 'ES2',
 'F',
 'FNM',
 'FNP',
 'GRAUPEL_ACC_NC',
 'HGT',
 'ISLTYP',
 'IVGTYP',
 'LAKEMASK',
 'LANDMASK',
 'LU_INDEX',
 'MAPFAC_M',
 'MAPFAC_MX',
 'MAPFAC_MY',
 'MAPFAC_U',
 'MAPFAC_UX',
 'MAPFAC_UY',
 'MAPFAC_V',
 'MAPFAC_VX',
 'MAPFAC_VY',
 'MAX_MSTFX',
 'MAX_MSTFY',
 'MF_VX_INV',


In [15]:
display(requests.get(dataset_url + '/info').json())

{'dimensions': {'time': 15336,
  'y': 1015,
  'x': 1367,
  'bottom_top_stag': 51,
  'bottom_top': 50,
  'soil_layers_stag': 4,
  'x_stag': 1368,
  'y_stag': 1016,
  'snow_layers_stag': 3,
  'snso_layers_stag': 7},
 'variables': {'ACDEWC': {'type': 'float32',
   'dimensions': ['time', 'y', 'x'],
   'attributes': {'description': 'acccumlated QDEWC',
    'grid_mapping': 'crs',
    'integration_length': '24-hour accumulation',
    'long_name': 'Accumulated canopy dew rate',
    'units': 'mm',
    'coordinates': 'lat lon'}},
  'ACDRIPR': {'type': 'float32',
   'dimensions': ['time', 'y', 'x'],
   'attributes': {'description': 'acccumlated QDRIPR',
    'grid_mapping': 'crs',
    'integration_length': '24-hour accumulation',
    'long_name': 'Accumulated canopy precipitation drip rate',
    'units': 'mm',
    'coordinates': 'lat lon'}},
  'ACDRIPS': {'type': 'float32',
   'dimensions': ['time', 'y', 'x'],
   'attributes': {'description': 'acccumlated QDRIPS',
    'grid_mapping': 'crs',
    'i

## Test OPeNDAP endpoints

`Catalog-To-Xpublish` is extendable using any `xpublish` plugin. In this example we use the [`xpublish_opendap`](https://github.com/xpublish-community/xpublish-opendap).

The following endpoints are added to the application:
* `datasets/{dataset_id}/opendap.dds` - returns the OPeNDAP Dataset Descriptor Structure (DDS) response.
* `datasets/{dataset_id}/opendap.das` - returns the OPeNDAP Data Attribute Structure (DAS) response.
* `datasets/{dataset_id}/opendap.dods` - returns the OPeNDAP Data Object Structure (DODS) response.

Additionally one can use OPeNDAP "constraint expressions" to sub-select data on the server side.


First we will test the `.dds` and `.das` endpoints. In this case we are getting "accumulated evapo-transpiration" data from the "zarr-s3" dataset ("ACETLSM").

In [16]:
opendap_url = dataset_url + '/opendap'

# here we can see a shape summary of the ACETLSM
print(requests.get(opendap_url + '.dds?ACETLSM').text)

Dataset {
    Grid {
      Array:
        Float32 ACETLSM[time = 15336][y = 1015][x = 1367];
      Maps:
        Float64 time[time = 15336];
        Float64 y[y = 1015];
        Float64 x[x = 1367];
    } ACETLSM;
} zarr-s3;



In [72]:
# here we can show an example of sub-setting the ACETLSM
time_range = '[100:150]'
y_range = '[100:300]'
x_range = '[100:300]'
subset_string = f'{time_range}{y_range}{x_range}'
print(requests.get(opendap_url + f'.dds?ACETLSM{subset_string}').text)

Dataset {
    Grid {
      Array:
        Float32 ACETLSM[time = 51][y = 201][x = 201];
      Maps:
        Float64 time[time = 51];
        Float64 y[y = 201];
        Float64 x[x = 201];
    } ACETLSM;
} zarr-s3;



In [73]:
# here we can see units and full attribute metadata
print(requests.get(opendap_url + '.das?ACETLSM').text)

Attributes {
    ACETLSM {
        String description "acccumlated ET";
        String grid_mapping "crs";
        String integration_length "24-hour accumulation";
        String long_name "Accumulated total evaporation";
        String units "mm";
    }
    Int32 AER_ANGEXP_OPT 1;
    Float64 AER_ANGEXP_VAL 1.2999999523162842;
    Int32 AER_AOD550_OPT 1;
    Float64 AER_AOD550_VAL 0.11999999731779099;
    Int32 AER_ASY_OPT 1;
    Float64 AER_ASY_VAL 0.8999999761581421;
    Int32 AER_OPT 1;
    Int32 AER_SSA_OPT 1;
    Float64 AER_SSA_VAL 0.8500000238418579;
    Int32 AER_TYPE 1;
    Float64 BLDT 0.0;
    Int32 BL_PBL_PHYSICS 1;
    Int32 BOTTOM-TOP_GRID_DIMENSION 51;
    Int32 BOTTOM-TOP_PATCH_END_STAG 51;
    Int32 BOTTOM-TOP_PATCH_END_UNSTAG 50;
    Int32 BOTTOM-TOP_PATCH_START_STAG 1;
    Int32 BOTTOM-TOP_PATCH_START_UNSTAG 1;
    Float64 BUCKET_J 1000000000.0;
    Float64 BUCKET_MM 100.0;
    Float64 CEN_LAT 39.100006103515625;
    Float64 CEN_LON -97.89999389648438;
    Float64 

## Test OPeNDAP clients

With a variable and sub-selection of data choses, we can now use the `.dods` response (binary) to open the data in `xarray` and as a `NetCDF`.

In [75]:
dods_url = opendap_url + f'?ACETLSM{subset_string}'
dods_url

'http://127.0.0.1:8000/conus404-daily/datasets/zarr-s3/opendap?ACETLSM[100:150][100:300][100:300]'

We start by getting our Zarr chunking information from the API.

In [76]:
zarr_url = dataset_url + '/zarr/.zmetadata'
zarr_url

'http://127.0.0.1:8000/conus404-daily/datasets/zarr-s3/zarr/.zmetadata'

In [77]:
zarr_dict = dict(requests.get(zarr_url).json())
chunks = zarr_dict['metadata'][f'ACETLSM/.zarray']['chunks']

In [78]:
dims_dict = dict(requests.get(dataset_url + '/info').json())['dimensions']
chunks_dict = {}
for i, c in enumerate(chunks):
    chunks_dict[list(dims_dict.keys())[i]] = c
chunks_dict


{'time': 36, 'y': 350, 'x': 350}

### `xarray`

In [113]:
# get the chunked data in xarray
xr_dataset = xr.open_dataset(
    dods_url,
    engine='netcdf4',
    chunks=chunks_dict,
)
xr_dataset

<xarray.Dataset>
Dimensions:  (time: 51, y: 201, x: 201)
Dimensions without coordinates: time, y, x
Data variables:
    ACETLSM  (time, y, x) float32 dask.array<chunksize=(36, 201, 201), meta=np.ndarray>

In [114]:
xr_dataset.ACETLSM.data

dask.array<open_dataset-ACETLSM, shape=(51, 201, 201), dtype=float32, chunksize=(36, 201, 201), chunktype=numpy.ndarray>

In [118]:
xr_dataset.ACETLSM[0].hvplot.image()

:Image   [x,y]   (ACETLSM)

### `netCDF4`

In [120]:
nc_dataset = nc.Dataset(dods_url)

In [124]:
nc_dataset.data_model

'NETCDF3_CLASSIC'

In [125]:
nc_dataset.dimensions

{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 51,
 'x': <class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 201,
 'y': <class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 201}

In [123]:
nc_dataset.variables

{'ACETLSM': <class 'netCDF4._netCDF4.Variable'>
 float32 ACETLSM(time, y, x)
 unlimited dimensions: 
 current shape = (51, 201, 201)
 filling off}